In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [2]:
#start training
def compute_accuracy(v_x, v_y):
    global pred
    #input v_x to nn and get the result with y_pre
    y_pre = sess.run(pred, feed_dict={x:v_x})
    #find how many right
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_y,1))
    #calculate average
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    #get input content
    result = sess.run(accuracy,feed_dict={x: v_x, y: v_y})
    return result

In [3]:
def Bi_lstm(X):
    lstm_f_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
    lstm_b_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
    return tf.contrib.rnn.static_bidirectional_rnn(lstm_f_cell, lstm_b_cell, X, dtype=tf.float32)

In [4]:
def RNN(X,weights,biases):
    # hidden layer for input
    X = tf.reshape(X, [-1, n_inputs])
    X_in = tf.matmul(X, weights['in']) + biases['in']

    #reshape data put into bi-lstm cell
    X_in = tf.reshape(X_in, [-1,n_steps, n_hidden_units])
    #hape_b=sess.run(X_in)
    #print("the shape before is ",shape_b.shape)
    #print(shape_b)
    X_in = tf.transpose(X_in, [1,0,2])
    #shape_a=sess.run(X_in)
    #print("the shape after is ",shape_a.shape)
    X_in = tf.reshape(X_in, [-1, n_hidden_units])
    X_in = tf.split(X_in, n_steps)
    outputs, _, _ = Bi_lstm(X_in)
    
    #hidden layer for output as the final results
    results = tf.matmul(outputs[-1], weights['out']) + biases['out']

    return results

# 超参数

In [5]:
#load mnist data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# parameters init
l_r = 0.001
training_iters = 1000000
batch_size = 128

n_inputs = 28
n_steps = 28
n_hidden_units = 128
n_classes = 10

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [6]:
#define placeholder for input
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

# define w and b
weights = {
    'in': tf.Variable(tf.random_normal([n_inputs,n_hidden_units])),
    'out': tf.Variable(tf.random_normal([2*n_hidden_units,n_classes]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1,shape=[n_hidden_units,])),
    'out': tf.Variable(tf.constant(0.1,shape=[n_classes,]))
}

In [7]:
pred = RNN(x, weights, biases)
soft_out=tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y)
cost = tf.reduce_mean(soft_out)
train_op = tf.train.AdamOptimizer(l_r).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [8]:
# x_image,x_label = mnist.test.next_batch(500)
# x_image = x_image.reshape([500, n_steps, n_inputs])
#init session
sess = tf.Session()
#init all variables
sess.run(tf.global_variables_initializer())
with tf.device("/cpu:0"):
    for i in range(5000):
        #get batch to learn easily
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape([batch_size, n_steps, n_inputs])
        p,y1,t,co,so=sess.run([pred,y,train_op,cost,soft_out],feed_dict={x: batch_x, y: batch_y})
        if i % 5 == 0:
            print(sess.run(accuracy,feed_dict={x: batch_x, y: batch_y,}))

0.132813
0.539063
0.5625
0.523438
0.695313
0.734375
0.664063
0.757813
0.773438
0.796875
0.828125
0.78125
0.789063
0.757813
0.8125
0.828125
0.859375
0.820313
0.835938
0.875
0.90625
0.851563
0.851563
0.890625
0.882813
0.828125
0.875
0.851563
0.828125
0.851563
0.890625
0.914063
0.914063
0.9375
0.898438
0.914063
0.960938
0.9375
0.890625
0.914063
0.929688
0.835938
0.914063
0.914063
0.9375
0.914063
0.890625
0.890625
0.921875
0.914063
0.898438
0.921875
0.945313
0.890625
0.953125
0.882813
0.953125
0.9375
0.96875
0.914063
0.90625
0.960938
0.9375
0.921875
0.96875
0.90625
0.914063
0.929688
0.945313
0.9375
0.914063
0.929688
0.921875
0.929688
0.96875
0.921875
0.929688
0.929688
0.929688
0.898438
0.9375
0.929688
0.96875
0.921875
0.929688
0.9375
0.976563
0.96875
0.960938
0.945313
0.9375
0.96875
0.945313
0.960938
0.976563
0.953125
0.90625
0.945313
0.953125
0.96875
0.96875
0.960938
0.976563
0.960938
0.953125
0.960938
0.945313
0.96875
0.960938
0.960938
0.984375
0.945313
0.96875
0.976563
0.929688
0.9375
0

In [9]:
test_data = mnist.test.images.reshape([-1, n_steps, n_inputs])
test_label = mnist.test.labels
#print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))
print("Testing Accuracy: ", compute_accuracy(test_data, test_label))

Testing Accuracy:  0.9817
